In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
data = pd.read_excel('dataset.xlsx',sheet_name='Explanations -ve').iloc[:576,:]  #or iloc 192
data_cols = data.columns
drop_cols = ['actual label','label','pregnant_val','Glucose_val','bp_val','skin_val','insulin_val','bmi_val','pedigree_val','age_val',]
#drop_cols= ['label','pregnant_val','Glucose_val','bp_val','skin_val','insulin_val','bmi_val','pedigree_val','age_val',]
data = data.drop(drop_cols,axis=1)

In [3]:
ot = pd.read_excel('annotations_formatted.xlsx')
ot.fillna(0,inplace = True)
ot

,glucose > 140.00,insulin <= 79.80,age > 40.25,skin > 32.00,32.00 < bmi <= 36.50,0.38 < pedigree <= 0.63,3.00 < pregnant <= 6.00,64.00 < bp <= 72.00,glucose <= 99.00,bmi <= 27.50,...,pedigree <= 0.24,79.80 < insulin <= 129.25,bmi > 36.50,insulin > 129.25,117.00 < glucose <= 140.00,99.00 < glucose <= 117.00,72.00 < bp <= 80.00,24.00 < age <= 29.00,1.00 < pregnant <= 3.00,bp > 80.00
0,her glucose levels are dangerously high,her insulin levels show a dip,she falls in the indicative age category,her skin fold thickness is high,her body mass index is high,the patient's pedigree indicates a nominal cha...,she has been pregnant many times,her diastolic blood pressure is normal,her glucose levels are decreasing,she has normal weight,...,the pedigree of the patient indicates low chan...,she has normal insulin levels,she is overweight,she has high insulin levels,the patient has high blood glucose levels,the patient's glucose level fall within normal...,the patient has normal blood pressure,the patient is a young adult,she has concieved a child one to three times,she has high resting diastolic blood pressure
1,her glucose levels are shooting,her body is not producing insulin properly,she is an old age patient,the patient has tight skin,her body mass index is sufficiently high,0,the patient has been pregnant more than 3 times,The blood pressure of the patient lies in the ...,the patient's glucose levels are very low,the patient has a normal bmi,...,0,the insulin levels of the patient are normal,the bmi of the patient is high,insulin levels of the patient are high,the patient's blood glucose levels fall in the...,patient's glucose level is in healthy range,the patient's blood pressure is in the healthy...,the patient is in her late twenties,the patient has been pregrant at least once bu...,the patient has high blood pressure
2,The glucose level falls in the high range,Insulin levels of the patient are low,The patient's age surpasses 40,the patient has a thick skin fold,The bmi of the patient falls in the high range,0,the patient has had more than three pregnancie...,the patient has normal blood pressure,the patient's glucose levels are low,the patient's BMI is normal,...,0,her body is producing enough insulin,bmi value indicates severly obese patient,her body is producing more insulin than required,0,0,0,0,0,the patient suffers from high blood pressure
3,the patient has high glucose levels,the patient has low insulin,the patient is aged more than 40 years,skin thickness is high,the patient's BMI indicates signs of obesity,0,number of pregnancies are more than 3,blood pressure is normal,0,0,...,0,0,patient is severly obese,0,0,0,0,0,0,0
4,blood tests report high glucose,blood tests indicate less insulin the patient'...,age is more than 40 years,0,BMI of the patient is recorded as high,0,0,0,0,0,...,0,0,patient's bmi values falls within severly obes...,0,0,0,0,0,0,0
5,0,0,0,0,BMI falls within obesity range,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#making explaination dictionary
explaination_dictionary = {}
for col in ot.columns:
    explaination_dictionary[col] = []

for col in ot.columns:
    column = ot.loc[:,col]
    for entry in column:
        if entry != 0:
            explaination_dictionary[col].append(entry)                            

In [6]:
def explain(reason,num):
    explaination = ""
    if(num==2):
        explaination+= " and "
    entry = random.randint(1,25) % len(explaination_dictionary[reason])
    random_entry_of_this_reason = explaination_dictionary[reason][entry].lower()
    explaination+= random_entry_of_this_reason
    return explaination

In [7]:
base_sentence = ['The patient is absolutely fine because ','The patient is predicted healthy because ','The patient is not at a risk of having diabetes because ']
#base_sentence = ['The patient has chances of diabetes because ','The patient is adviced to take diabeties medicine because ','The patient is at a risk of having diabetes because ']

In [8]:
explainations = []
for row in range(data.shape[0]):   #data.shape[0]
    #print("row number: " + str(row))
    for sentence_number in range(2):
        thisExplaination = base_sentence[random.randint(0,2)]
        thisrow = data.iloc[row,:]
        top1 = thisrow['Top 1']
        top2 = thisrow['Top 2']

        if(top1 == 0 or top1 ==np.nan or top1==1):
            top1=-1
            thisExplaination+=" ERROR ERROR ERROR"
            #print('error invoked for entry {number}'.format(number=row))
            explainations.append(thisExplaination)
            continue
        if(top2 == 0 or top2 ==np.nan or top2 == 1):
            top2=-1


        reason1 = thisrow[thisrow == top1].index[1]
        thisExplaination+= explain(reason1,1)

        if(top2 == -1):
            explainations.append(thisExplaination)
            continue
        try:
            reason2 = thisrow[thisrow == top2].index[1]
            if 2*row+sentence_number==119:
                print(reason2)
        except:
            print('problem')
        thisExplaination+= explain(reason2,2)
        explainations.append(thisExplaination)

pedigree <= 0.24


In [9]:
len(explainations)

1152

In [12]:
exp1 = []
exp2 = []

for i in range(len(explainations)):
    if i%2==0:
        exp1.append(explainations[i])
    else:
        exp2.append(explainations[i])

In [14]:
data['explanation1']=exp1
data['explanation2']=exp2

In [15]:
data.to_csv('explanations.csv',index=False)